# Leitura do dataset

In [1]:
import pandas as pd

df = pd.read_csv('dodf_atos_pessoal_v1.csv', encoding = 'latin-1')
df

,id_ato,id_dodf,num_doc_dodf,data_doc_dodf,tipo_rel,id_rel,anotador_rel,texto_rel,tipo_ent,id_ent,anotador_ent,texto_ent
0,34_224.24.10.2014-R5,34_224.24.10.2014,224,24.10.2014,Ato_Nomeacao_Comissionado,R5,lygia_paloma,NOMEAR MARIA GERLANIA SOARES para exercer o Ca...,simbolo,4678,DODFMiner,DFA-14
1,34_224.24.10.2014-R5,34_224.24.10.2014,224,24.10.2014,Ato_Nomeacao_Comissionado,R5,lygia_paloma,NOMEAR MARIA GERLANIA SOARES para exercer o Ca...,orgao,4676,DODFMiner,Governadoria do Distrito Federal
2,34_224.24.10.2014-R5,34_224.24.10.2014,224,24.10.2014,Ato_Nomeacao_Comissionado,R5,lygia_paloma,NOMEAR MARIA GERLANIA SOARES para exercer o Ca...,nome,4677,DODFMiner,MARIA GERLANIA SOARES
3,34_224.24.10.2014-R5,34_224.24.10.2014,224,24.10.2014,Ato_Nomeacao_Comissionado,R5,lygia_paloma,NOMEAR MARIA GERLANIA SOARES para exercer o Ca...,hierarquia_lotacao,4650,DODFMiner,"Assessoria, do Gabinete, da Administracao Regi..."
4,34_224.24.10.2014-R5,34_224.24.10.2014,224,24.10.2014,Ato_Nomeacao_Comissionado,R5,lygia_paloma,NOMEAR MARIA GERLANIA SOARES para exercer o Ca...,cargo_comissionado,7956,lygia_paloma,Assessor
...,...,...,...,...,...,...,...,...,...,...,...,...
36035,83_216.30.10.8-R231,83_216.30.10.8,216,30.10.8,Ato_Substituicao,R231,marco_antonio,"DESIGNAR DURVALINA SILVA RABELO, matricula 26....",simbolo_objeto_substituicao,7107,marco_antonio,DFG-04
36036,83_216.30.10.8-R231,83_216.30.10.8,216,30.10.8,Ato_Substituicao,R231,marco_antonio,"DESIGNAR DURVALINA SILVA RABELO, matricula 26....",orgao,7110,marco_antonio,Procuradoria-Geral do Distrito Federal
36037,83_216.30.10.8-R231,83_216.30.10.8,216,30.10.8,Ato_Substituicao,R231,marco_antonio,"DESIGNAR DURVALINA SILVA RABELO, matricula 26....",data_inicial,7111,marco_antonio,16/10
36038,83_216.30.10.8-R231,83_216.30.10.8,216,30.10.8,Ato_Substituicao,R231,marco_antonio,"DESIGNAR DURVALINA SILVA RABELO, matricula 26....",data_final,7112,marco_antonio,23/\n10/2008


# Criação de funções de obtenção de labels

In [2]:
try:
    from dodflib.core.preprocess import IOBifyer
except ModuleNotFoundError:
    from dodflib.core.preprocess import IOBifyer
from dodflib.core.feature_extraction import Tokenizer

In [3]:
def filtra_ato(df, ato):
    df_ato = df.loc[df['tipo_rel'] == ato]
    df_ato.reset_index(drop = True)
    df_ato = df_ato.filter(['id_ato', 'tipo_ent', 'texto_ent'])
    df_ato = df_ato.reset_index(drop = True)
    act_types = df['tipo_rel'].value_counts()
    return df_ato, act_types
    
def obter_entidades(df, act_types):
    
    row_list = []

    id_atual = df['id_ato'][0]
    d = {}

    for row in df.iterrows():
        id_ato = row[1]['id_ato']
        tipo_ent = row[1]['tipo_ent']
        texto_ent = row[1]['texto_ent']

        if tipo_ent in act_types:
            tipo_ent = 'text'

        if id_ato != id_atual:
            row_list.append(d)
            d = {}
            id_atual = id_ato

        d[tipo_ent] = texto_ent
    
    return pd.DataFrame(row_list)

def obter_IOB(df):
    
    text_notnull = df['text'].dropna()
    df = df.loc[text_notnull.index]
    df = df.reset_index(drop = True)
    
    tokenizer = Tokenizer()
    labels_list = []

    text_index = 0
    for i in df.columns:
        if i == 'text':
            break
        text_index += 1
    
    for index, row in df.iterrows():
        labels = IOBifyer.generate_IOB_labels(row, text_index, tokenizer)
        labels_list.append(labels)
        assert len(labels) == len(tokenizer(row['text'])), 'index %d: %d != %d' %(index, len(labels), len(tokenizer(row['text'])))

    df['labels'] = labels_list
    df['labels'] = df['labels'].apply(lambda x: ' '.join(x))
    return df

#### Teste

In [4]:
df_ato, act_types = filtra_ato(df, 'Ato_Substituicao')
df_ato = obter_entidades(df_ato, act_types)
df_com_labels = obter_IOB(df_ato)

In [5]:
df_com_labels

,nome_substituto,matricula_substituto,cargo_substituto,simbolo_substituto,hierarquia_lotacao,orgao,data_inicial,data_final,motivo,nome_substituido,matricula_substituido,cargo_objeto_susbtituicao,simbolo_objeto_substituicao,text,matricula_SIAPE,labels
0,ALCIDES GOMES DE ARAUJO FILHO,1.655.773-5,"Gerente,",DFG-14,"Diretoria de Administracao Geral,","Administracao Regional do Lago \nSul, da Casa ...",1o,10 de setembro de 2014,ferias e sem \nprejuizo de suas atribuicoes,"KARLA MENDES BATISTA,",1.660.492-X,Diretora,CNE-07,"DESIGNAR ALCIDES GOMES DE ARAUJO FILHO, matric...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
1,EDUARDO DE MIRANDA DUARTE,16533844,Encarregado do Nucleo de Material e Patrimonio,DFG-05,NaN,"Administracao Regional do Riacho Fundo, \nda C...",10/11/2014,09/12/2014,ferias de regulamentar do titular.,"LUIZ CARLOS DE SA,",16607406,Diretor de Administracao Geral,CNE-07,"DESIGNAR EDUARDO DE MIRANDA DUARTE, matricula ...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
2,CARMEM LUCIA DE CARVALHO SANTOS,40.579-5,Tecnico Juridico,NaN,Diretor da Diretoria de Arrecadacao \nTributar...,Procuradoria Geral do Distrito Federal,16/10/2014,"25/10/2014,",licenca medica do titular,MARCELO DE OLIVEIRA PEREIRA,34.437-0,"Natureza Especial,",CNE-07,"DESIGNAR CARMEM LUCIA DE CARVALHO SANTOS, matr...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
3,ROSINEIDE RODRIGUES MUNIZ,41.616-9,Agente \nJuridico,NaN,Assessor da Assessoria Executiva da Coordenaca...,"Procuradoria Geral do Distrito Federal,",28/10/2014,06/11/2014,ferias regulamentares da titular.,KELLY DE SOUZA MORAIS,168.642-9,Comissao,DFA-14,"DESIGNAR ROSINEIDE RODRIGUES MUNIZ, matricula ...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
4,DARIO BALBINO DA SILVA,91.994-2,NaN,NaN,"Gerencia de Suprimentos e Servicos Gerais, da ...",Secretaria de Estado de Obras,06/10/2008,04/11/\n2008,ferias do titular,SEBASTIAO\nABDALA,162.088-6,Chefe,DFG-09,"DESIGNAR DARIO BALBINO DA SILVA, matricula 91....",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,HELDER PAIS DE OLIVEIRA,137.542-3,CE Enfermeiro,NaN,"Gerencia de Enferma-\ngem, da Diretoria Geral ...",NaN,29.12 .2008,17.01.2009,ferias do Titular,VENERY RODRIGUES GALVAO,NaN,Supervisor de Enfermagem,DFG-05,"DESIGNAR HELDER PAIS DE OLIVEIRA, CE Enfermei...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
1243,ZANA MARIA SILVELLO,134.496-X,Nutricionista,NaN,Nu-\ncleo de Nutricao e Dietetica,Hospital Regional do Paranoa,08/12/2008,27/12/\n2008,virtude de ferias,MARJORIE LIMA BRITO FERREIRA,154.225-7,chefia,NaN,"DESIGNAR ZANA MARIA SILVELLO, matricula 134.49...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
1244,AMILTON LOIOLA MENEZES,0000233-X,NaN,NaN,Gerencia de Pessoal/CAO/ FE-\nPECS,NaN,12.11.08,26.11.08,ferias da titular,CELIA\nMILHOMEM AMARAL,0080927-6,Gerente,DFG-11,"DESIGNAR AMILTON LOIOLA MENEZES, matricula 000...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
1245,EDSON MOULIN PEDROSA,0084933-2,NaN,NaN,Gerencia de Atividades Gerais/\nCAO/FEPECS,NaN,24.11.08,13.12.08,ferias do\ntitular,RUBENS\nBATISTA DE OLIVEIRA,0084373-3,Gerente,DFG-11,"DESIGNAR EDSON MOULIN PEDROSA, matricula 00849...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...


In [17]:
def obter_labels(df, ato):
    df_ato, act_types = filtra_ato(df, ato)
    df_ato = obter_entidades(df_ato, act_types)
    df_com_labels = obter_IOB(df_ato)
    return df_com_labels

# Criação das funções de Classificação

In [7]:
import pandas as pd
import sklearn_crfsuite

In [8]:
def tokenizar_labels(df):
    tokenizer = Tokenizer()
    data = df
    x = []
    y = []
    for row in range(len(data)):
        if pd.notna(data['text'][row]) and pd.notna(data['labels'][row]):
            x.append(tokenizer(data['text'][row]))
            y.append(data['labels'][row].split())
    return x, y

def get_features_(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
def get_features(x):
    for i in range(len(x)):
        x[i] = get_features_(x[i])
    
def split_train_test(x, y):
    import math
    split_idx = math.floor(0.8*len(x))
    x_train = x[0:split_idx]
    y_train = y[0:split_idx]
    x_test = x[split_idx:]
    y_test = y[split_idx:]
    return x_train, x_test, y_train, y_test

def classificacao(x_train, x_test, y_train, y_test):
    import sklearn_crfsuite
    from sklearn_crfsuite import metrics

    crf = sklearn_crfsuite.CRF(
        algorithm = 'l2sgd',
        c2=1,
        max_iterations=100,
        all_possible_transitions=True,
        verbose = False,
    )

    crf.keep_tempfiles = False
    crf.model_filename = None
    
    crf.fit(x_train, y_train)
    
    classes = list(crf.classes_)
    classes.remove('O')

    y_pred = crf.predict(x_test)
    score = metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)
    
    print('f1-score: %f' %score)

    print(metrics.flat_classification_report(
        y_test, y_pred, labels=classes, digits=3
    ))
    
    return score

#### Teste

In [9]:
x, y = tokenizar_labels(df_com_labels)
get_features(x)
x_train, x_test, y_train, y_test = split_train_test(x, y)
score = classificacao(x_train, x_test, y_train, y_test)

f1-score: 0.802125


/home/felipe/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-nome_substituto', 'I-nome_substituto', 'B-matricula_substituto', 'I-matricula_substituto', 'B-cargo_substituto', 'B-simbolo_substituto', 'I-simbolo_substituto', 'B-orgao', 'I-orgao', 'B-data_inicial', 'B-data_final', 'I-data_final', 'B-motivo', 'I-motivo', 'B-nome_substituido', 'I-nome_substituido', 'B-cargo_objeto_susbtituicao', 'B-simbolo_objeto_substituicao', 'I-simbolo_objeto_substituicao', 'B-hierarquia_lotacao', 'I-hierarquia_lotacao', 'I-cargo_substituto', 'B-matricula_substituido', 'I-cargo_objeto_susbtituicao', 'I-data_inicial', 'I-matricula_substituido', 'B-matricula_SIAPE'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/felipe/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Pre

                               precision    recall  f1-score   support

            B-nome_substituto      0.684     0.736     0.709       250
            I-nome_substituto      0.673     0.729     0.700       735
       B-matricula_substituto      0.913     0.771     0.836       271
       I-matricula_substituto      0.875     0.822     0.848       484
           B-cargo_substituto      0.793     0.689     0.737       267
         B-simbolo_substituto      0.923     0.343     0.500        35
         I-simbolo_substituto      0.923     0.500     0.649        24
                      B-orgao      0.862     0.892     0.877       203
                      I-orgao      0.919     0.941     0.930      1198
               B-data_inicial      0.959     0.928     0.943       250
                 B-data_final      0.942     0.974     0.958       235
                 I-data_final      0.915     0.977     0.945       698
                     B-motivo      0.971     0.967     0.969       240
     

/home/felipe/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/felipe/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
def classificar_com_crf(df):
    x, y = tokenizar_labels(df)
    get_features(x)
    x_train, x_test, y_train, y_test = split_train_test(x, y)
    return classificacao(x_train, x_test, y_train, y_test)

# Treinamento de todos os modelos

In [23]:
tipos_de_ato = df['tipo_rel'].value_counts().keys()

scores = {}

In [24]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [25]:
for ato in tipos_de_ato:
    print("Ato: %s" %ato)
    scores[ato] = classificar_com_crf(obter_labels(df, ato))

Ato: Ato_Substituicao
f1-score: 0.802936
                               precision    recall  f1-score   support

            B-nome_substituto      0.688     0.740     0.713       250
            I-nome_substituto      0.677     0.733     0.704       735
       B-matricula_substituto      0.935     0.742     0.827       271
       I-matricula_substituto      0.899     0.793     0.843       484
           B-cargo_substituto      0.831     0.663     0.737       267
         B-simbolo_substituto      0.923     0.343     0.500        35
         I-simbolo_substituto      0.923     0.500     0.649        24
                      B-orgao      0.869     0.852     0.861       203
                      I-orgao      0.945     0.909     0.927      1198
               B-data_inicial      0.959     0.944     0.952       250
                 B-data_final      0.950     0.970     0.960       235
                 I-data_final      0.938     0.974     0.956       698
                     B-motivo      

f1-score: 0.558880
                        precision    recall  f1-score   support

      B-tipo_documento      0.900     0.500     0.643        18
         B-numero_dodf      0.647     0.786     0.710        14
           B-data_dodf      0.812     0.684     0.743        19
           I-data_dodf      0.812     0.765     0.788        68
         B-pagina_dodf      0.917     0.733     0.815        15
            B-tipo_ato      0.765     0.619     0.684        21
             B-lotacao      0.833     0.500     0.625        10
             I-lotacao      0.640     0.667     0.653        48
B-informacao_corrigida      0.889     0.545     0.676        44
I-informacao_corrigida      0.345     0.888     0.497       197
      B-data_documento      0.857     0.667     0.750        18
      I-data_documento      0.857     0.727     0.787        66
                B-nome      0.000     0.000     0.000        13
                I-nome      0.000     0.000     0.000        33
         I-pagina_do

f1-score: 0.835832
                      precision    recall  f1-score   support

              B-nome      1.000     0.500     0.667         6
              I-nome      0.889     0.941     0.914        17
         B-matricula      1.000     0.667     0.800         6
         I-matricula      1.000     0.692     0.818        13
     B-cargo_efetivo      1.000     0.600     0.750         5
     I-cargo_efetivo      0.864     0.864     0.864        22
B-hierarquia_lotacao      0.000     0.000     0.000         0
I-hierarquia_lotacao      0.000     0.000     0.000         0
             B-orgao      1.000     1.000     1.000         3
             I-orgao      1.000     1.000     1.000        21
  B-fundamento_legal      1.000     1.000     1.000         7
  I-fundamento_legal      0.595     1.000     0.746        47
   B-a_pedido_ou_nao      1.000     0.833     0.909        12
   I-a_pedido_ou_nao      1.000     0.500     0.667         6
          B-carreira      0.000     0.000     0.00

In [27]:
for ato in scores:
    print("%s: %f" %(ato, scores[ato]))

Ato_Substituicao: 0.802936
Ato_Nomeacao_Comissionado: 0.967928
Ato_Exoneracao_Comissionado: 0.914823
Ato_Retificacao_Efetivo: 0.863009
Ato_Retificacao_Comissionado: 0.558880
Ato_Tornado_Sem_Efeito_Exo_Nom: 0.913576
Ato_Cessao: 0.733839
Ato_Abono_Permanencia: 0.792570
Ato_Nomeacao_Efetivo: 0.325756
Ato_Exoneracao_Efetivo: 0.835832
Ato_Reversao: 0.093590
Ato_Tornado_Sem_Efeito_Apo: 0.559460
